
## EXERCICE 8 : Utilisation de Pandas et sklearn pour l'analyse de données réelles



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing


### Question 1 : 
- utiliser la fonction pandas.read_csv avec les bonnes options pour charger le fichier RealMedicalData.csv. On remarquera que le separateur dans le fichier csv est ';' et que les decimales sont representees par des virgules et non des points. Il est important de remarquer qu'un dataframe et non un numpy array sera retourne. Cette classe contient l'equivalent d'un numpy.array mais aussi le nom des lignes et colonnes, tout comme dans les dataframes de R
- Extraire deux numpy.arrays X et y des donnes pour faire par la suite de la regression. Nous allons essayer d'expliquer  'Disease progression' avec les autres variables. Ce sera alors la colonne qui contient y. Les autres colonnes contiendront les donnees de X.
- Une fois X et y extrait, centrer et reduire X. La fonction sklearn.preprocessing.scale pourra etre utilisee


In [55]:
df = pd.read_csv('data/RealMedicalData.csv', sep=';', decimal=',')

In [56]:
df.head()

,Acid 1 density,Acid 2 density,Acid 3 density,Cells 1 density,Cells 2 density,Pressure 1,Pressure 2,Bio-marker 1,Bio-marker 2,Biomarker 3,Biomarker 4,Biomarker 5,Biomarker 6,Biomarker 6.1,Biomarker 7,Biomarker 8,Biomarker 9,Biomarker 10,Disease progression
0,11.6,6.4,4.3,7.2,14.4,19.26,39.32,50.00,13.040000,0.20,0.0440,2.44,2.209,9.02,0.26,1.23,3.56,16.91,6
1,15.5,10.5,4.9,6.9,13.7,8.57,36.78,22.57,18.713333,0.51,0.8501,2.32,0.339,22.43,0.10,0.56,7.06,-3.47,6
2,15.3,6.5,4.2,6.4,13.6,13.48,25.90,8.25,8.246667,0.27,0.9629,2.83,0.339,17.94,0.13,0.59,3.20,10.05,10
3,13.4,7.7,4.1,5.8,14.2,15.23,44.36,11.09,11.093333,0.67,-0.2756,5.06,1.611,11.38,0.20,0.88,4.33,-3.62,18
4,16.0,5.2,3.2,5.7,12.8,8.37,19.62,7.44,6.386667,0.24,-0.6928,1.59,0.659,8.56,0.27,1.20,4.57,1.56,7


In [57]:
y = np.array(df['Disease progression'])
X = np.array(df.drop(columns='Disease progression'))
n = len(X)
X

array([[11.6 ,  6.4 ,  4.3 , ...,  1.23,  3.56, 16.91],
       [15.5 , 10.5 ,  4.9 , ...,  0.56,  7.06, -3.47],
       [15.3 ,  6.5 ,  4.2 , ...,  0.59,  3.2 , 10.05],
       ...,
       [21.  , 10.3 ,  7.  , ...,  0.76,  2.37,  1.97],
       [11.  ,  4.8 ,  3.86, ...,  0.51,  2.34,  2.11],
       [18.5 ,  5.  ,  3.49, ...,  0.48,  4.01,  1.99]])

In [61]:
X = preprocessing.scale(X)
X

array([[-0.39452241,  0.32973066,  0.13599835, ...,  0.58355627,
        -0.40971102,  1.80051593],
       [ 0.53841345,  3.06250605,  0.78879044, ..., -0.84133927,
         1.80003087, -0.60474976],
       [ 0.49057058,  0.39638372,  0.02719967, ..., -0.77753797,
        -0.63699875,  0.99089264],
       ...,
       [ 1.85409222,  2.92919993,  3.07356275, ..., -0.41599732,
        -1.16102326,  0.03728387],
       [-0.538051  , -0.73671827, -0.34271585, ..., -0.94767475,
        -1.1799639 ,  0.0538068 ],
       [ 1.25605642, -0.60341216, -0.74527097, ..., -1.01147604,
        -0.12560135,  0.03964429]])

In [62]:
thresh = n // 2
X_train = X[thresh:]
y_train = y[thresh:]
X_test = X[:thresh]
y_test = y[:thresh]


### Question 2 : 
- Mettre en lien X et y a l'aide de la regression lineaire multiple avec une penalisation Lasso
- A la vue des resultats, quelle variable vous semble liee a 'Disease progression'?
- Verifier cette relation en representant des nuages de points mettant en lien les observations de chaque variable avec les observations de 'Disease progression'



In [67]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Définir les valeurs d'alpha à tester sur une échelle exponentielle large
alphas = np.logspace(-3, 2, 100)

kf = KFold(n_splits=8, shuffle=True, random_state=42)

lasso_regressor_kf = LassoCV(alphas=alphas, cv=kf, random_state=42)
lasso_regressor_kf.fit(X_train, y_train)

y_pred_lasso_kf = lasso_regressor_kf.predict(X_test)
mse_score_lasso = mean_squared_error(y_test, y_pred_lasso_kf)
print(f"MSE: {mse_score_lasso}")

# Afficher la MSE moyenne pour chaque alpha testé
mse_per_alpha = lasso_regressor_kf.mse_path_.mean(axis=1)  # MSE pour chaque alpha à travers les folds
for i, alpha in enumerate(lasso_regressor_kf.alphas_):
    if(alpha == lasso_regressor_kf.alpha_):
        print(f"Meilleur alpha: {alpha}, MSE moyenne: {mse_per_alpha[i]}")


MSE: 18.584978015586522
Meilleur alpha: 0.298364724028334, MSE moyenne: 9.704428119530663
